In [ ]:
import os
import cv2
import mediapipe as mp

In [ ]:
img_size = 224
frame_rate = 5

In [ ]:
dataset_dir = "/home/mostafabakr/Desktop/Project X/img data"  
video_dataset_dir = "/home/mostafabakr/Desktop/Project X/vid data"  
frame_output_dir = "/home/mostafabakr/Desktop/Project X/Test_img/frames"  

In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

In [ ]:
def detect_hand(image, padding=150):
    """Detects a hand in the image and returns a wider cropped version around the hand."""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        height, width, _ = image.shape
        hand_landmarks = results.multi_hand_landmarks[0]
        x_coords = [int(point.x * width) for point in hand_landmarks.landmark]
        y_coords = [int(point.y * height) for point in hand_landmarks.landmark]
        x_min = max(0, min(x_coords) - padding)
        x_max = min(width, max(x_coords) + padding)
        y_min = max(0, min(y_coords) - padding)
        y_max = min(height, max(y_coords) + padding)
        return image[y_min:y_max, x_min:x_max]
    return None

In [ ]:
def preprocess_with_hand_detection(dataset_dir):
    """Preprocess the dataset to extract hands from images."""
    hand_dir = f"{dataset_dir}_hand"
    os.makedirs(hand_dir, exist_ok=True)

    #24 Iterations
    for label_dir in os.listdir(dataset_dir):
        label_path = os.path.join(dataset_dir, label_dir)
        hand_label_dir = os.path.join(hand_dir, label_dir)
        os.makedirs(hand_label_dir, exist_ok=True)
        
        #3000 - 4000 Iterations according to images
        for image_file in os.listdir(label_path):
            image_path = os.path.join(label_path, image_file)
            image = cv2.imread(image_path)
            hand = detect_hand(image)
            if hand is not None:
                hand_resized = cv2.resize(hand, (img_size, img_size))
                cv2.imwrite(os.path.join(hand_label_dir, image_file), hand_resized)

    print("Hand detection and preprocessing complete.")

In [ ]:
def preprocess_frames_with_hand_detection(video_dataset_dir, output_dir):
    """Preprocess video frames to extract hands and save them."""
    os.makedirs(output_dir, exist_ok=True)

    for label_dir in os.listdir(video_dataset_dir):
        label_path = os.path.join(video_dataset_dir, label_dir)
        output_label_dir = os.path.join(output_dir, label_dir)
        os.makedirs(output_label_dir, exist_ok=True)

        for video_file in os.listdir(label_path):
            video_path = os.path.join(label_path, video_file)
            cap = cv2.VideoCapture(video_path)
            frame_count = 0

            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                if frame_count % frame_rate == 0:
                    hand = detect_hand(frame)
                    if hand is not None:
                        hand_resized = cv2.resize(hand, (img_size, img_size))
                        frame_filename = os.path.join(
                            output_label_dir, f"{os.path.splitext(video_file)[0]}_frame_{frame_count}.jpg"
                        )
                        cv2.imwrite(frame_filename, hand_resized)
                frame_count += 1

            cap.release()

    print("Hand frame extraction complete.")

In [ ]:
"""Choose One or Both"""
preprocess_with_hand_detection(dataset_dir)
preprocess_frames_with_hand_detection(video_dataset_dir, frame_output_dir)